In [1]:
from new_model import * 

In [2]:
seed = 10
random.seed(seed)
np.random.seed(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(seed)
if device == 'cuda':
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
#Data 다운로드 경로지정

batch_size = 128


#Data Process 
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_val = transforms.Compose([transforms.ToTensor(), 
                                    transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

transform_test = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

train_CIFAR10 = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)

test_CIFAR10 = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)

num_train = int(1.0 * len(train_CIFAR10) * 95 / 100)
num_val = len(train_CIFAR10) - num_train
train_CIFAR10, val_CIFAR10 = torch.utils.data.random_split(train_CIFAR10, [num_train, num_val])

train_loader = torch.utils.data.DataLoader(train_CIFAR10, batch_size=batch_size, shuffle=True, num_workers=2) #num_workers는 데이터 로드시 sub process 몇개 쓸거냐 
val_loader = torch.utils.data.DataLoader(val_CIFAR10, batch_size=batch_size,shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_CIFAR10, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
#CIFAR 100 : Data 다운로드 경로지정
batch_size = 128

#Data Process
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_val = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_test = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

train_CIFAR100 = torchvision.datasets.CIFAR100(root='../data', train=True, download=True, transform=transform_train)

test_CIFAR100 = torchvision.datasets.CIFAR100(root='../data', train=False, download=True, transform=transform_test)

num_train = int(1.0 * len(train_CIFAR100) * 95 / 100)
num_val = len(train_CIFAR100) - num_train
train_CIFAR100, val_CIFAR100 = torch.utils.data.random_split(train_CIFAR100, [num_train, num_val])

train_loader = torch.utils.data.DataLoader(train_CIFAR100, batch_size=batch_size, shuffle=True, num_workers=2) #num_workers는 데이터 로드시 sub process 몇개 쓸거냐 
val_loader = torch.utils.data.DataLoader(val_CIFAR100, batch_size=batch_size,shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_CIFAR100, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
def node_prune(model, pruning_ratio):
    for layer in model.modules():
        if isinstance(layer, M_BasicBlock):
            # calculate cutoff value
            nonzero_vals = layer.get_2norm()[0][layer.M_relu1.active != 0]
            cutoff1 = torch.quantile(nonzero_vals, pruning_ratio)

            # set inactive nodes to 0
            layer.M_relu1.active.data[layer.get_2norm()[0] < cutoff1] *= 0
            
            # calculate cutoff value
            nonzero_vals = layer.get_2norm()[1][layer.M_relu2.active != 0]
            cutoff2 = torch.quantile(nonzero_vals, pruning_ratio)

            # set inactive nodes to 0
            layer.M_relu2.active.data[layer.get_2norm()[1] < cutoff2] *= 0

In [5]:
## scheduler 학습
def fine_tuning(model, mode, EPOCHS):
    # total number of training epochs
    CHECKPOINT_PATH = "./save_seed10_CIFAR100"

    best_val_acc = 0
    current_learning_rate = INITIAL_LR
    
    remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    print(f'최초 node의 갯수 : {remaining_node1, remaining_node2}')    

    print("==> Training starts!")
    start = time.time()
    optimizer = torch.optim.SGD(model.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=REG)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones = [60, 120, 160], gamma = 0.2)
#     scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=0)
    for i in range(EPOCHS):
        if i % 40 == 0 :
            print(f'{(i//40)+1}번 자르고 fine tuning을 시작합니다.')
            node_prune(model, ratio)
            
            remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
            remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])

            print(f'{(i//40)+1}번 pruning 이후 남은 node 는 : {(remaining_node1, remaining_node2)}')
        
        '''
        train loop
        '''            
        model.train()
        
        # this help you compute the training accuracy
        total_examples = 0
        correct_examples = 0

        train_loss = 0 # track training loss if you want

        # Train the model for 1 epoch.
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device).float(), targets.to(device).long() #inputs과 targets는 gpu로 계산

            # compute the output and loss
            y_preds = model(inputs)        
            loss = criterion(y_preds,targets)
            train_loss += loss.item()

            # zero the gradient
            optimizer.zero_grad()

            # backpropagation
            loss.backward()

            # apply gradient and update the weights
            optimizer.step()

            # count the number of correctly predicted samples in the current batch
            y_preds_class = torch.argmax(y_preds, dim=1)
            correct_examples += (targets == y_preds_class).sum().item()
            total_examples += targets.size(0)

        scheduler.step()    #스케쥴러 사용해보기
            
        avg_loss_tr = train_loss / len(train_loader) ###
        avg_acc_tr = correct_examples / total_examples ### 

                                                      
        '''
        validation loop
        '''
                                                      
        # switch to eval mode
        model.eval()
        total_examples = 0
        correct_examples = 0
        val_loss = 0 # again, track the validation loss if you want
        
        # disable gradient during validation, which can save GPU memory
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(val_loader):
                inputs, targets = inputs.to(device).float(), targets.to(device).long()

                # compute the output and loss
                y_preds = model(inputs)        
                loss = criterion(y_preds,targets)
                val_loss += loss.item()

                # count the number of correctly predicted samples in the current batch
                y_preds_class = torch.argmax(y_preds, dim=1)
                correct_examples += (targets == y_preds_class).sum().item()
                total_examples += targets.size(0)

        avg_loss_val = val_loss / len(val_loader)
        avg_acc_val = correct_examples / total_examples

                                                      
        # save the model checkpoint
        if avg_acc_val > best_val_acc:
            best_val_acc = avg_acc_val
            if not os.path.exists(CHECKPOINT_PATH):
                os.makedirs(CHECKPOINT_PATH)

            state = {'state_dict': model.state_dict(),
                     'epoch': i,
                     'lr': current_learning_rate}
            
            torch.save(state, os.path.join(CHECKPOINT_PATH, '{}.pth'.format(mode)))        
    
        if i % 10 == 9 :  
            end = time.time()
            diff_time = round(end - start,2)
            print("Epoch %d:" %(i+1), f"progress time is {diff_time} sec")
            print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss_val, avg_acc_val))
    print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")                       

    return model

In [6]:
###8. test_model
def test_model(model):
    model.to(device)
    model.eval()

    total_examples = 0
    correct_examples = 0
    softmax = torch.nn.Softmax(dim=1)

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            pred = model(inputs)
            total_examples += inputs.shape[0]

            out = softmax(pred)
            out = torch.max(out, 1)

            correct_examples += torch.sum(targets==out[1]).cpu().data.numpy().tolist()

    avg_acc = correct_examples / total_examples
    print("Total examples is {}, correct examples is {}; Test accuracy: {}".format(total_examples, correct_examples, avg_acc))

## 모델지정

In [7]:
INITIAL_LR = 0.1*0.2*0.2

MOMENTUM = 0.9

REG = 5e-4

criterion = nn.CrossEntropyLoss()

ratio = 1-0.5**0.194

mode = '28x4_multistepLR_lr0.1*0.2*0.2'

In [8]:
# 모델 지정 및 GPU 지정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Wide_ResNet(28, 4, 0.3, 100).to(device)
check_prune = torch.load("../1.Wide_ResNet_model/saved_model/WR_28x4_cifar100.pth")
model.load_state_dict(check_prune['state_dict'])
# test_model(model)

| Wide-Resnet 28x4


<All keys matched successfully>

In [21]:
##scheduler 실행 : multstpepLR / 0.1*0.2*0.2
fine_tuning(model, mode = '28x4_CIFAR10_scheduler_multstep_lr0.1*0.2', EPOCHS = 200)

최초 node의 갯수 : (tensor(1792., device='cuda:3'), tensor(1792., device='cuda:3'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(1564., device='cuda:3'), tensor(1564., device='cuda:3'))
Epoch 10: progress time is 507.96 sec
Validation loss: 0.2967, Validation accuracy: 0.9072
Epoch 20: progress time is 1015.81 sec
Validation loss: 0.2875, Validation accuracy: 0.9148
Epoch 30: progress time is 1522.95 sec
Validation loss: 0.3529, Validation accuracy: 0.8984
Epoch 40: progress time is 2032.13 sec
Validation loss: 0.3090, Validation accuracy: 0.9136
41번 자르고 fine tuning을 시작합니다.
41번 pruning 이후 남은 node 는 : (tensor(1368., device='cuda:3'), tensor(1368., device='cuda:3'))
Epoch 50: progress time is 2538.77 sec
Validation loss: 0.3104, Validation accuracy: 0.9084
Epoch 60: progress time is 3044.43 sec
Validation loss: 0.2491, Validation accuracy: 0.9248
Epoch 70: progress time is 3547.96 sec
Validation loss: 0.1856, Validation accuracy: 0.9480
Epoch 80: progress 

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3)

In [9]:
##scheduler 실행 : cosineannealingLR
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(1792., device='cuda:2'), tensor(1792., device='cuda:2'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(1564., device='cuda:2'), tensor(1564., device='cuda:2'))
Epoch 10: progress time is 534.36 sec
Validation loss: 1.0176, Validation accuracy: 0.7500
Epoch 20: progress time is 1066.07 sec
Validation loss: 1.0235, Validation accuracy: 0.7476
Epoch 30: progress time is 1600.4 sec
Validation loss: 1.0461, Validation accuracy: 0.7424
Epoch 40: progress time is 2131.72 sec
Validation loss: 1.0671, Validation accuracy: 0.7316
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(1368., device='cuda:2'), tensor(1368., device='cuda:2'))
Epoch 50: progress time is 2660.77 sec
Validation loss: 1.1724, Validation accuracy: 0.7092
Epoch 60: progress time is 3188.72 sec
Validation loss: 1.1975, Validation accuracy: 0.7172
Epoch 70: progress time is 3725.21 sec
Validation loss: 1.0151, Validation accuracy: 0.7656
Epoch 80: progress tim

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3)

In [9]:
##scheduler 실행 : multistepLR, lr = 0.1*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(1792., device='cuda:2'), tensor(1792., device='cuda:2'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(1564., device='cuda:2'), tensor(1564., device='cuda:2'))
Epoch 10: progress time is 529.76 sec
Validation loss: 1.5045, Validation accuracy: 0.6348
Epoch 20: progress time is 1069.4 sec
Validation loss: 1.2802, Validation accuracy: 0.6860
Epoch 30: progress time is 1604.16 sec
Validation loss: 1.3266, Validation accuracy: 0.6672
Epoch 40: progress time is 2142.22 sec
Validation loss: 1.2811, Validation accuracy: 0.6832
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(1368., device='cuda:2'), tensor(1368., device='cuda:2'))
Epoch 50: progress time is 2680.68 sec
Validation loss: 1.2299, Validation accuracy: 0.6820
Epoch 60: progress time is 3218.29 sec
Validation loss: 1.2811, Validation accuracy: 0.6720
Epoch 70: progress time is 3762.29 sec
Validation loss: 0.9576, Validation accuracy: 0.7640
Epoch 80: progress tim

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3)

## seed 10

In [9]:
##scheduler 실행 : multistepLR, lr = 0.1*0.2*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(1792., device='cuda:2'), tensor(1792., device='cuda:2'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(1564., device='cuda:2'), tensor(1564., device='cuda:2'))
Epoch 10: progress time is 525.3 sec
Validation loss: 0.0267, Validation accuracy: 0.9924
Epoch 20: progress time is 1059.81 sec
Validation loss: 0.0196, Validation accuracy: 0.9944
Epoch 30: progress time is 1596.36 sec
Validation loss: 0.0455, Validation accuracy: 0.9848
Epoch 40: progress time is 2133.78 sec
Validation loss: 0.0427, Validation accuracy: 0.9868
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(1368., device='cuda:2'), tensor(1368., device='cuda:2'))
Epoch 50: progress time is 2666.09 sec
Validation loss: 0.0973, Validation accuracy: 0.9696
Epoch 60: progress time is 3199.36 sec
Validation loss: 0.1167, Validation accuracy: 0.9640
Epoch 70: progress time is 3729.53 sec
Validation loss: 0.0433, Validation accuracy: 0.9860
Epoch 80: progress tim

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3)

In [9]:
##cifar100
##scheduler 실행 : multistepLR, lr = 0.1*0.2*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(1792., device='cuda:0'), tensor(1792., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(1564., device='cuda:0'), tensor(1564., device='cuda:0'))
Epoch 10: progress time is 466.68 sec
Validation loss: 0.1994, Validation accuracy: 0.9460
Epoch 20: progress time is 941.08 sec
Validation loss: 0.2337, Validation accuracy: 0.9332
Epoch 30: progress time is 1414.83 sec
Validation loss: 0.3154, Validation accuracy: 0.9084
Epoch 40: progress time is 1889.06 sec
Validation loss: 0.4139, Validation accuracy: 0.8888
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(1368., device='cuda:0'), tensor(1368., device='cuda:0'))
Epoch 50: progress time is 2362.98 sec
Validation loss: 0.7615, Validation accuracy: 0.8020
Epoch 60: progress time is 2836.12 sec
Validation loss: 0.7642, Validation accuracy: 0.7924
Epoch 70: progress time is 3310.51 sec
Validation loss: 0.5065, Validation accuracy: 0.8608
Epoch 80: progress tim

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3)

## 결과 확인

In [23]:
## cifar10
check_prune = torch.load("./saved_tuning/28x4_CIFAR10_scheduler_multstep.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

Total examples is 10000, correct examples is 9526; Test accuracy: 0.9526


In [24]:
## cifar10
check_prune = torch.load("./saved_tuning/28x4_CIFAR10_scheduler_multstep_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

send_email('28x4_CIFAR10_scheduler_multstep lr0.1*0.2 finished')

Total examples is 10000, correct examples is 9505; Test accuracy: 0.9505


## CIFAR100

In [10]:
## cifar100
check_prune = torch.load("./saved_tuning_CIFAR100/28x4_multistepLR_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

send_email('28x4_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7581; Test accuracy: 0.7581


In [10]:
## cifar100
check_prune = torch.load("./saved_tuning_CIFAR100/28x4_multistepLR_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x4_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7609; Test accuracy: 0.7609


## seed 10 

In [10]:
## cifar10
check_prune = torch.load("./save_seed10_CIFAR10/28x4_multistepLR_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 9457; Test accuracy: 0.9457


In [10]:
## cifar10
check_prune = torch.load("./save_seed10_CIFAR10/28x4_multistepLR_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 9504; Test accuracy: 0.9504


In [9]:
## cifar10
check_prune = torch.load("./save_seed10_CIFAR10/28x4_multistepLR_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 9504; Test accuracy: 0.9504


In [11]:
## cifar10
check_prune = torch.load("./save_seed10_CIFAR10/28x4_cosine_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 9514; Test accuracy: 0.9514


### cifar100

In [10]:
## cifar100
check_prune = torch.load("./save_seed10_CIFAR100/28x4_multistepLR_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7461; Test accuracy: 0.7461
